**TensorFlow** is an open source software library for numerical computation using data flow graphs.

**TensorFlow Basics** - TensorFlow program consists of two phases
- Phase-1: Assemble a graph
- Phase-2: Use a session to execute operations in the graph

This may change in future with experimental [TensorFlow's eager mode](https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html)

**What is a Tensor?**
- An n-dimensional array
    - 0-d tensor: scalar
    - 1-d tensor: vector
    - 2-d tensor: matrix
    
**Data Flow Graphs**
- NODES: operators, variables and constants (TF automatically names the nodes if the name is not explicitly provided)
- EDGES: tensors

**`tf.Session()`** - A Session object encapsulates the environment in which `Operation` objects are executed, and `Tensor` objects are evaluated. Session will also allocate memory to store the current values of variables.

**Resources**
- [TensorFlow Official Documentation](https://www.tensorflow.org/api_docs/)
- [TensorFlow Official Sample Models](https://github.com/tensorflow/models)

In [1]:
# Imports
import tensorflow as tf

In [2]:
# Simple graph
a = tf.add(2, 3)
print a

Tensor("Add:0", shape=(), dtype=int32)


In [3]:
# Evaluate graph to fetch values
sess = tf.Session()
print sess.run(a)
sess.close()

5


In [4]:
# Using with
with tf.Session() as sess:
    print sess.run(a)

5


In [5]:
# Graph
x = 2
y = 3

add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
pow_op = tf.pow(mul_op, add_op)

with tf.Session() as sess:
    print sess.run(pow_op)

7776


In [6]:
# Sub-Graph
x = 2
y = 3

add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless_op = tf.multiply(x, add_op) 
pow_op = tf.pow(mul_op, add_op) # pow_op does not depend on useless_op so session won't compute useless_op

with tf.Session() as sess:
    print sess.run(pow_op)

7776


```python
# Pass all variables whose values need to be fetched in a list or tuple (check help)
tf.Session.run(fetches, feed_dict=None, options=None, run_metadata=None)
```

`fetches` - List of tensors whose values you want

In [7]:
# Sub-Graph
x = 2
y = 3

add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless_op = tf.multiply(x, add_op) 

# pow_op does not depend on useless_op so session won't compute useless_op -> save computation
pow_op = tf.pow(mul_op, add_op) 

with tf.Session() as sess:
    fetches = (pow_op, useless_op)
    pow_out, not_useless = sess.run(fetches)
    print pow_out, not_useless

7776 10


```python
# Distributed Computation

# Create a graph
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], name='a')
    b = tf.constant([5.0, 4.0, 3.0, 2.0, 1.0], name='b')
    c = tf.multiply(a, b)
    
# Creates a session with log_device_placement set to True
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print sess.run(c)
sess.close()
```

In [8]:
# Distributed Computation

# Create a graph
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], name='a')
    b = tf.constant([5.0, 4.0, 3.0, 2.0, 1.0], name='b')
    c = tf.multiply(a, b)
    
# Creates a session with log_device_placement set to True
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print sess.run(c)
sess.close()

[5. 8. 9. 8. 5.]


**Multiple Graphs: AVOID!** otherwise there will be BUGs
- The session runs the default graph
- No need for more than one graph. Multiple graphs require multiple sessions, each will try to use all available resources by default.

In [9]:
# Create a graph
g = tf.Graph()

# To add operations to a graph, set it as default
with g.as_default():
    x = tf.add(2, 3)
    
with tf.Session(graph=g) as sess:
    print sess.run(x)

5


In [10]:
# Handling default graph
g = tf.get_default_graph()
print g

In [11]:
# Better way: Do not mix default graph and user created graphs
g1 = tf.get_default_graph()
g2 = tf.Graph()
print(g1, g2)

# Add operations to default graph
with g1.as_default():
    a = tf.constant(3)
    
# Add operations to user created graph
with g2.as_default():
    b = tf.constant(5)
    
with tf.Session(graph=g1) as sess:
    print sess.run(a)

with tf.Session(graph=g2) as sess:
    print sess.run(b)

(<tensorflow.python.framework.ops.Graph object at 0x7fc9c4025b90>, <tensorflow.python.framework.ops.Graph object at 0x7fc9b96f3190>)
3
5
